In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, udf
from pyspark.sql.types import DoubleType

import matplotlib.pyplot as plt
from constants import LOGISTICS_DATA, MATERIALS_DATA, PROJECTS_DATA, SUPPLIERS_DATA, DATA, FINAL_DATA_parquet

In this notbeook I do some durther preprocessing and data engineering. There are still questions I want to answer about the final data:

- Data Summary: A statistical summary of the dataset's numerical columns (mean, median, standard deviation, min, max)
- Cardinality of Categorical Features: Knowing how many unique values some of the categorical features have (e.g., transport_mode, project_location, supplier_location) can inform the feature engineering steps
- Correlations: Understanding the correlation between the features, especially with the target variable (CO2_emissions) if it's a supervised learning problem

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CO2 Emission ML Pipeline - Data Preprocessing") \
    .getOrCreate()


DATA SUMMARY

In [6]:
#reading df
final_df = spark.read.parquet(FINAL_DATA_parquet)

final_df.show(n=10)

+-----------+----------+------------------+--------------+----------------+--------+--------------+------------------+------------------+------------+------------------+----------------+----------------+-----------+-------------+-----------------+-------------+-----------------+---------------+
|material_id|project_id|    project_budget|transaction_id|transaction_date|quantity|transport_mode|  distance_covered|      CO2_emission|project_name|project_start_date|project_end_date|project_location|supplier_id|material_name|material_category|supplier_name|supplier_location|supplier_rating|
+-----------+----------+------------------+--------------+----------------+--------+--------------+------------------+------------------+------------+------------------+----------------+----------------+-----------+-------------+-----------------+-------------+-----------------+---------------+
|        290|        32|248950.37356851666|             1|      2020-01-01|       2|          Rail|168.038611626

In [5]:
# Calculate summary statistics
summary_stats = final_df.describe()

# Fetch the statistics for each column from the summary DataFrame
means = {row['summary']: row.asDict() for row in summary_stats.collect()}["mean"]
maxs = {row['summary']: row.asDict() for row in summary_stats.collect()}["max"]
mins = {row['summary']: row.asDict() for row in summary_stats.collect()}["min"]
stddevs = {row['summary']: row.asDict() for row in summary_stats.collect()}["stddev"]

# calculate median for each numerical column
medians = {}
for column in final_df.columns:
    # Check if column is numerical by trying to cast it to a double; skip if casting fails (essentially changing the var type)
    try:
        final_df = final_df.withColumn(column, final_df[column].cast('double'))
        medians[column] = final_df.approxQuantile(column, [0.5], 0.0)[0]
    except Exception as e:
        continue

# Display stats for each column
for column in final_df.columns:
    if column in means:  # This checks if the column is numerical (present in the summary stats)
        print(f"Statistics for {column}:")
        print(f"Mean: {means[column]}")
        print(f"Max: {maxs[column]}")
        print(f"Min: {mins[column]}")
        print(f"Standard Deviation: {stddevs[column]}")
        print(f"Median: {medians.get(column, 'N/A')}")
        print("--------------")


Statistics for material_id:
Mean: 496.76872549019606
Max: 1000.0
Min: 1.0
Standard Deviation: 290.96310745073055
Median: 497.0
-----------------------
Statistics for project_id:
Mean: 50.42980392156863
Max: 100.0
Min: 1.0
Standard Deviation: 28.75723049387397
Median: 51.0
-----------------------
Statistics for project_budget:
Mean: 295840.8306163639
Max: 1950581.5483643755
Min: 53805.64013990858
Standard Deviation: 209570.71331281099
Median: 270037.4610087137
-----------------------
Statistics for transaction_id:
Mean: 5006.130980392157
Max: 10000.0
Min: 1.0
Standard Deviation: 2884.331836773124
Median: 5009.0
-----------------------
Statistics for transaction_date:
Mean: None
Max: None
Min: None
Standard Deviation: None
Median: N/A
-----------------------
Statistics for quantity:
Mean: 50.396470588235296
Max: 99.0
Min: 1.0
Standard Deviation: 28.470351337471453
Median: 50.0
-----------------------
Statistics for transport_mode:
Mean: None
Max: None
Min: None
Standard Deviation: None
M

CARDINALITY: number of unique values for each of the specified categorical columns

In [9]:
categorical_columns = ["transport_mode", "project_location", "supplier_location"]

for column in categorical_columns:
    distinct_count = final_df.select(column).distinct().count()
    print(f"Cardinality of {column}: {distinct_count}")

Cardinality of transport_mode: 3
Cardinality of project_location: 4
Cardinality of supplier_location: 3


CORRELATION BETWEEN NUMERICAL FEATURES AND TARGET FEATURE CO2_EMISSIONS

In [15]:
# List of features
features = final_df.columns
features.remove('CO2_emission')  # Remove the target variable

# Calculate correlations with the target variable
correlations = [(feature, final_df.stat.corr(feature, 'CO2_emission')) for feature in features if final_df.select(feature).dtypes[0][1] in ('double', 'int')]

for feature in correlations:
    print(f"The correlation between CO2_emissions and {feature[0]} is: {feature[1]}")

The correlation between CO2_emissions and material_id is: 0.013309843960719402
The correlation between CO2_emissions and project_id is: 0.015389057139567884
The correlation between CO2_emissions and project_budget is: -0.008704442492230928
The correlation between CO2_emissions and transaction_id is: 0.011334588931078056
The correlation between CO2_emissions and quantity is: 0.5558180227071717
The correlation between CO2_emissions and distance_covered is: 0.21013827019533382
The correlation between CO2_emissions and supplier_id is: -0.01578169754900137
The correlation between CO2_emissions and supplier_rating is: 0.00353117673371521
